### Importing libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.api import VAR, DynamicVAR, VARMAX
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import Holt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pyramid.arima import auto_arima


In [2]:
# Seasonal MASE script
# Define seasonality as 12 (monthly data) in argument when ground truth data includes at least 24 months

def seasonal_MASE(truth, forecast, seasonality=1):    
    period = truth.shape[0] # T
    # print(period)
    forecast_errors = np.abs(truth - forecast)
    # print(forecast_errors)
    mean_absolute_forecast_error = np.sum(forecast_errors) / period
    # print(mean_absolute_forecast_error)

    naive_period = truth.shape[0] - seasonality # T - m
    # print(naive_period)
    # print(truth[seasonality:])
    # print(truth[:period - seasonality])
    naive_errors = np.abs(truth[seasonality:] - truth[:period - seasonality])
    mean_absolute_naive_error = np.sum(naive_errors) / naive_period
    
    return mean_absolute_forecast_error / mean_absolute_naive_error

### Dataset


In [5]:
nights = pd.read_csv('/Users/jinny/Documents/touristcast/datasets/nights_2010-2017.csv', index_col='date', parse_dates=['date'], 
                          usecols=[*range(1, 15)])
temp = pd.read_csv('/Users/jinny/Documents/touristcast/datasets/avgtemp_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])
daysoff = pd.read_csv('/Users/jinny/Documents/touristcast/datasets/daysoff_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=['date','daysoff'])
gdp = pd.read_csv('/Users/jinny/Documents/touristcast/datasets/regionalGDP_2010-2015.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])

# df with data for training set
data_PAC = pd.concat([nights, temp, daysoff, gdp], axis=1)
data_PAC = data_PAC.dropna()
data_PAC = data_PAC[['nights_PAC','avgtemp_PAC','gdp_PAC','daysoff']]

# df with nights only for testing set
nights_PAC = nights[['nights_PAC']]

### Stationarity + Seasonality
- **Do checks apart** using scripts written before (see previous VAR tests)
- See if series already stationary: If not stationary, do seasonal_decompose
- Seasonality: Multiplicative or Additive
- Adjust according in 'for' loop below (seasonal_decompose step + forecast step)

## Gigantic CV 'for' loop
1. Create train and test datasets (+ formatting for later)
2. Seasonal decompose
3. Predictions for: 
    - Seasonal (copypaste of last year)
    - Trend (Holt)
    - Residuals (VAR)
4. Recompose for forecast
5. Calculated error per CV step

In [9]:
# Make sure to fill these in for each region! Input data must have datetime as index

input_data = data_PAC         # dataframe-type dataset
target = nights_PAC           # dataframe-type dataset
start_year = '2010-01-01'     # date as string

for index in range(2012, 2018):
    end_year = str(index)+'-01-01'
    predict_year = str(index+1)+'-01-01'
    
    training_period = (input_data.index>=start_year)&(input_data.index<end_year) 
    testing_period = (target.index>=end_year)&(target.index<predict_year)
    
    train_data = input_data[training_period]
    test_data = np.array(target[testing_period]) # Array format to calculate errors at the end
    
    
    # --------Seasonal decompose----------
    
    # - IMPORTANT - Change model (multiplicative or additive) based on season type of nights
    decomposed_data = seasonal_decompose(train_data, model='multiplicative')  

    seasonal_data = decomposed_data.seasonal.dropna()
    trend_data = decomposed_data.trend.dropna()
    residual_data = decomposed_data.resid.dropna()
    
    # ---------Seasonal prediction (Same as last year of training data)---------

    # Duplicate last year's seasonal data
    seasonal_forecast = seasonal_data[(seasonal_data.index>=start_year)&(seasonal_data.index<end_year)]
    
    # Forecasted DF without datetime index and only 12 months to be able to recompose later
    # - IMPORTANT - Replace nights_PAC below with column name for your region
    seasonal_forecast_df = pd.DataFrame(seasonal_forecast.nights_PAC.values[-12:])
    
    # --------Trend prediction (Linear Holt)----------
    
    # Holt model (w/ optimized fit) on nights
    # - IMPORTANT - Replace nights_PAC below with column name for your region
    trend_model = Holt(trend_data.nights_PAC).fit(optimized=True)
    
    # Predict 12 months of trend
    trend_forecast = trend_model.predict(start=0, end=11)
    
    # Forecasted DF without datetime index to be able to recompose later
    trend_forecast_df = pd.DataFrame(trend_forecast.values) 
    
    # --------Residual prediction (VAR)----------

    # VAR model: Don't use maxlags, use specific lags in fit argument below
    resid_model = VAR(residual_data, dates=residual_data.index)
    # - IMPORTANT - Replace lag number in fit below with desired lags
    resid_results = resid_model.fit(3)
    
    lag_order = resid_results.k_ar

    # Forecasted DF without datetime index to be able to recompose later
    resid_forecast_df = pd.DataFrame(resid_results.forecast(residual_data.values[-lag_order:], 12))

    # --------Recomposing results----------
    # - IMPORTANT - Addition if additive series /// Multiplication if multiplicative series
    forecast = seasonal_forecast_df[0] * trend_forecast_df[0] * resid_forecast_df[0]
    
    
    # --------Calculated error measures for each CV step----------
    rmse_test = np.sqrt(mean_squared_error(test_data, forecast))
    mae_test = mean_absolute_error(test_data, forecast)
    
    print('Years of training data:', train_data.index.strftime('%Y').unique().tolist())
    print('Predicted year:', end_year)
    print('VAR lag order:', lag_order)
    print('RMSE test:', rmse_test)
    print('MAE test:', mae_test)
    print('-------')

Years of training data: ['2010', '2011']
Predicted year: 2012-01-01
VAR lag order: 3
RMSE test: 104.43265967719107
MAE test: 94.37319052195699
-------
Years of training data: ['2010', '2011', '2012']
Predicted year: 2013-01-01
VAR lag order: 3
RMSE test: 110.45073857083092
MAE test: 82.60564105445503
-------
Years of training data: ['2010', '2011', '2012', '2013']
Predicted year: 2014-01-01
VAR lag order: 3
RMSE test: 101.07616603350503
MAE test: 76.42134629999482
-------
Years of training data: ['2010', '2011', '2012', '2013', '2014']
Predicted year: 2015-01-01
VAR lag order: 3
RMSE test: 73.89593366831619
MAE test: 57.44109927869453
-------
Years of training data: ['2010', '2011', '2012', '2013', '2014', '2015']
Predicted year: 2016-01-01
VAR lag order: 3
RMSE test: 101.21931418133535
MAE test: 85.34349859359948
-------
Years of training data: ['2010', '2011', '2012', '2013', '2014', '2015']
Predicted year: 2017-01-01
VAR lag order: 3
RMSE test: 138.1658432249903
MAE test: 111.538313